## Construcción de dataset de la serie del tiofen
(Antonio)

Para construir el dataset de esta serie se usaran los compuestos con ids del 26 al 42, el smile que le corresponde, la media de su ic50 y la desviación estándar de la misma.

Primeramente se construye el dataset con los compuestos del 26 al 37 ya que mediante los LLM son fáciles de detectar y obtener su SMILE, esto luego es validado mediatne rdkit. En el caso de los compuestos que se ven como grafos se dibujan usando el (https://www.cheminfo.org/Chemistry/Cheminformatics/FormatConverter/index.html), antes se experimentó con HyperChem (están en carpeta mol2_files), y luego se transforman a smile con rdkit. Al final se juntan todos los compuestos en un mismo dataset con los datos obtenidos de la tabla.

In [160]:
import pandas as pd # construcción del dataset
from rdkit import Chem # obtención de smile a partir del mol2
from rdkit.Chem import AllChem
import os # trabajo con ficheros

### Construcción de dataset inicial

In [161]:
thiophene_data = [
    {"id": 26, "SMILES": "O=[N+]([O-])c1cc(sc1)C"},
    {"id": 27, "SMILES": "O=[N+]([O-])c1cc(sc1)CC"},
    {"id": 28, "SMILES": "O=[N+]([O-])c1cc(sc1)C1CC1"},
    {"id": 29, "SMILES": "O=[N+]([O-])c1cc(sc1)CCCCO"},
    {"id": 30, "SMILES": "O=[N+]([O-])c1cc(sc1)CCCO"},
    {"id": 31, "SMILES": "O=[N+]([O-])c1cc(sc1)CC(CO)OC"},
    {"id": 32, "SMILES": "O=[N+]([O-])c1cc(sc1)CCCCCO"},
    {"id": 33, "SMILES": "O=[N+]([O-])c1cc(sc1)CCCN(CC)CC"},
    {"id": 34, "SMILES": "O=[N+]([O-])c1cc(sc1)CCCN(C)C"},
    {"id": 35, "SMILES": "O=[N+]([O-])c1cc(sc1)CCCCOCCC"},
    {"id": 36, "SMILES": "O=[N+]([O-])c1cc(sc1)CC=C"},
    {"id": 37, "SMILES": "O=[N+]([O-])c1cc(sc1)CC#C"},
]
df = pd.DataFrame(thiophene_data)
df.head()

,id,SMILES
0,26,O=[N+]([O-])c1cc(sc1)C
1,27,O=[N+]([O-])c1cc(sc1)CC
2,28,O=[N+]([O-])c1cc(sc1)C1CC1
3,29,O=[N+]([O-])c1cc(sc1)CCCCO
4,30,O=[N+]([O-])c1cc(sc1)CCCO


In [162]:
# Función para validar y canonicalizar
def validate_and_canonicalize(smiles):
    if not isinstance(smiles, str) or smiles.strip() == "":
        print ("empty")
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print ("invalid")
    canonical = Chem.MolToSmiles(mol, canonical=True)
    print ("valid")

# Mostrar resultados
print("=== Resultados de validación ===")
results = df["SMILES"].apply(lambda s: validate_and_canonicalize(s))


=== Resultados de validación ===
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid


### Obtención de compuestos representados mediante grafos

In [163]:
mol2_dir = "db/mol2_files"
new_rows = []

for filename in os.listdir(mol2_dir):
    if not filename.endswith(".ml2"):
        continue

    try:
        comp_id = int(filename.split(".")[0])
    except ValueError:
        print(f"⚠️  Nombre no numérico ignorado: {filename}")
        continue

    # Leer mol2
    mol2_path = os.path.join(mol2_dir, filename)
    mol = Chem.MolFromMol2File(mol2_path, sanitize=True, removeHs=False)

    if mol is None:
        print(f"❌ Fallo al leer {filename}")
        continue

    # Canonicalizar SMILES
    try:
        smiles = Chem.MolToSmiles(mol, canonical=True)
    except Exception as e:
        print(f"⚠️  Error al canonicalizar {comp_id}: {e}")
        continue

    # Crear nueva fila
    new_row = {
        "id": comp_id,
        "SMILES": smiles,
    }
    new_rows.append(new_row)

# Convertir a DataFrame
if new_rows:
    # Asegurar que las columnas coincidan con `df`
    df_new = df_new[df.columns]  # reordena según el orden de `df`
    df = pd.concat([df, df_new], ignore_index=True)
    print(f"✅ Añadidas {len(new_rows)} nuevas filas desde .mol2")
else:
    print("ℹ️  No se añadieron nuevas filas.")

✅ Añadidas 5 nuevas filas desde .mol2


In [164]:
# SMILES completos de los compuestos 38–42 (ya ensamblados)
complete_smiles = {
    38: "[N+](=O)(c1ccc(s1)c1nnc(s1)NCC1OCCCC1)[O-]",
    39: "[N+](=O)(c1ccc(s1)c1nnc(s1)NCc1ccccc1)[O-]",
    40: "[N+](=O)(c1ccc(s1)c1nnc(s1)NCc1ccc(cc1)OC)[O-]",
    41: "[N+](=O)(c1ccc(s1)c1nnc(s1)NCc1ncccc1)[O-]",
    42: "[N+](=O)(c1ccc(s1)c1nnc(s1)NCc1cnccc1)[O-]"
}

In [165]:
# Canonicalizar y validar los SMILES
canonical_smiles = {}
for comp_id, smi in complete_smiles.items():
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        print(f"❌ SMILES inválido para ID {comp_id}: {smi}")
        continue
    canonical = Chem.MolToSmiles(mol, canonical=True)
    canonical_smiles[comp_id] = canonical

# Actualizar las filas en df
updated_count = 0
for idx, row in df.iterrows():
    comp_id = row["id"]
    if comp_id in canonical_smiles:
        df.at[idx, "SMILES"] = canonical_smiles[comp_id]
        updated_count += 1

print(f"✅ Actualizados {updated_count} SMILES con estructuras completas.")

✅ Actualizados 5 SMILES con estructuras completas.


In [166]:
df.head()

,id,SMILES
0,26,O=[N+]([O-])c1cc(sc1)C
1,27,O=[N+]([O-])c1cc(sc1)CC
2,28,O=[N+]([O-])c1cc(sc1)C1CC1
3,29,O=[N+]([O-])c1cc(sc1)CCCCO
4,30,O=[N+]([O-])c1cc(sc1)CCCO


### Unión de ambos conjuntos de datos del tiofen

In [167]:
# IDs de la serie tiofeno (26–42, incluyendo 33 y 34)
thiophene_ids = [26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]
# IC50_mean (media) → comas decimales corregidas a puntos
IC50_mean = [64.0, 98.4, 97.3, 3.0, 104.0, 56.0, 3.0, 36.0, 25.0, 42.0, 34.3, 61.2, 35.8, 115.6, 198.0, 228.0, 156.0]
# IC50_std (desviación estándar)
IC50_std = [0.32, 0.12, 0.98, 0.41, 0.7, 0.74, 0.5, 0.24, 0.17, 0.45, 0.5, 0.22, 0.62, 0.63, 0.48, 0.9, 0.16]

In [168]:
# Crear un DataFrame con los valores correctos
df_ic50 = pd.DataFrame({
    'id': thiophene_ids,
    'IC50_mean': IC50_mean,
    'IC50_std': IC50_std
})

# Eliminar columna antigua si existe
if 'IC50' in df.columns:
    df = df.drop(columns=['IC50'])

# Unir por 'id'
df = df.merge(df_ic50, on='id', how='left')

In [169]:
df.head()

,id,SMILES,IC50_mean,IC50_std
0,26,O=[N+]([O-])c1cc(sc1)C,64.0,0.32
1,27,O=[N+]([O-])c1cc(sc1)CC,98.4,0.12
2,28,O=[N+]([O-])c1cc(sc1)C1CC1,97.3,0.98
3,29,O=[N+]([O-])c1cc(sc1)CCCCO,3.0,0.41
4,30,O=[N+]([O-])c1cc(sc1)CCCO,104.0,0.70


In [170]:
# exportar
df.to_csv("db/dataset_qsar_tiofeno.csv", index=False)

## Construcción serie furan
(Yonathan)

La representación en formato SMILES de las estructuras químicas se generó utilizando la herramienta deconversión de formatos disponible en la plataforma en línea Cheminfo ChemInfo.org (https://www.cheminfo.org/Chemistry/Cheminformatics/FormatConverter/index.html)(https://www.cheminfo.org/Chemistry/Cheminformatics/FormatConverter/index.html), 
específicamente el convertidor ubicado en la sección de Cheminformática (Cheminformatics/Format Converter). Esta herramienta permitió la conversión directa de las estructuras moleculares dibujadas interactivamenteal formato SMILES canónico, asegurando la correcta representación de la conectividad atómica y las propiedades estructurales de los compuestos en estudio. La plataforma Cheminfo, basada en las librerías quimioinformáticas RDKit y Open Babel, garantizó la generación de representaciones estandarizadas y válidas para el posterior análisis computacional

In [171]:
df_furan = pd.read_csv('db/compuestos.csv')
df_furan.head()

,no,smiles,IC50,desviacion
0,3.0,c1c(oc(c1)c1sc(nn1)NC)[N+](=O)[O-],54.0,0.17
1,4.0,c1c(oc(c1)c1sc(nn1)NCC)[N+](=O)[O-],50.0,0.80
2,5.0,c1c(oc(c1)c1sc(nn1)N1CC1)[N+](=O)[O-],55.5,0.66
3,6.0,c1c(oc(c1)c1sc(nn1)NCCO)[N+](=O)[O-],21.0,0.65
4,7.0,c1c(oc(c1)c1sc(nn1)NCCCO)[N+](=O)[O-],18.0,0.20


In [172]:
df_furan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   no          23 non-null     float64
 1   smiles      23 non-null     object 
 2   IC50        23 non-null     float64
 3   desviacion  23 non-null     float64
dtypes: float64(3), object(1)
memory usage: 1.4+ KB


In [173]:
# ajuste de dataset para vincular con el otro
df_furan = df_furan.rename(columns={
    "no": "id",
    "smiles": "SMILES",
    "IC50": "IC50_mean",
    "desviacion": "IC50_std"
})

df_furan["id"] = df_furan["id"].astype("Int64")
df_furan = df_furan.dropna(subset=["id", "SMILES"]).reset_index(drop=True)

In [174]:
# union de datasets
df = pd.concat([df_furan, df], ignore_index=True)
df.head()

,id,SMILES,IC50_mean,IC50_std
0,3,c1c(oc(c1)c1sc(nn1)NC)[N+](=O)[O-],54.0,0.17
1,4,c1c(oc(c1)c1sc(nn1)NCC)[N+](=O)[O-],50.0,0.80
2,5,c1c(oc(c1)c1sc(nn1)N1CC1)[N+](=O)[O-],55.5,0.66
3,6,c1c(oc(c1)c1sc(nn1)NCCO)[N+](=O)[O-],21.0,0.65
4,7,c1c(oc(c1)c1sc(nn1)NCCCO)[N+](=O)[O-],18.0,0.20


In [175]:
# Validar
print("=== Resultados de validación ===")
results = df["SMILES"].apply(lambda s: validate_and_canonicalize(s))

=== Resultados de validación ===
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid
valid


In [176]:
df.to_csv('db/smilesdf.csv', index=False)